# 🛡️Proyecto SafeStep: Asistente AR para Visión Reducida

Para ver el proyecto en su totalidad y no solo el notebook con la implementación y ejecución de este, puede acceder al siguiente repositorio en Github, que contiene los vídeos de entrada y salida, los audios con las órdenes y los generados por el asistente, así como las imágenes también de entrada y salida utilizadas. Además, dispone de un README.md que explica con detalle el flujo de trabajo llevado a cabo para la realización del SafeStep:

https://github.com/aos35/multimodal.git


**SafeStep** es un sistema de asistencia inteligente diseñado para aumentar la autonomía y seguridad de peatones con visión reducida en entornos urbanos. Utilizando técnicas avanzadas de **Visión Artificial y Realidad Aumentada (AR)**, el sistema procesa el entorno en tiempo real para identificar peligros, interpretar señales y ofrecer feedback multimodal (visual, auditivo y háptico).

### 🎯Objetivos Principales

El sistema busca resolver la dificultad de detectar elementos críticos en el tráfico (como el estado de un semáforo lejano o un coche silencioso acercándose) mediante:

1. **Detección de Peligros Dinámicos:** Monitorización constante de vehículos y peatones con alertas de colisión.

2. **Mejora Visual (Smart Zoom):** Una "Lupa Inteligente" que detecta, recorta y amplía automáticamente semáforos y señales lejanas.

3. **Navegación Segura:** Identificación de pasos de cebra y estimación de distancia a objetos.

4. **Interfaz de Alto Contraste:** UI accesible con códigos de color simplificados (🔴 Peligro / 🟢 Seguro) y textos de gran tamaño.

5. **Accesibilidad Multimodal**: Generación de avisos tanto visuales como sonoros para usuarios con baja visión o en situaciones de alta distracción.

6. **Personalización de Alertas**: Adaptación de la cantidad y tipo de avisos según el contexto o preferencias del usuario.

### 🛠️ Tecnologías Implementadas

Este prototipo integra un pipeline complejo de procesamiento de video:

- **YOLOv8 (Ultralytics):** Para la detección y tracking (seguimiento) de objetos en tiempo real.

- **OpenCV:** Para el procesamiento de imagen (CLAHE para visión nocturna, detección de líneas de carril, superposiciones gráficas).

- **EasyOCR:** Para el reconocimiento óptico de caracteres (OCR) en señales de tráfico y carteles.

- **Whisper (OpenAI):** Para la comprensión de comandos de voz del usuario.

- **gTTS (Google Text-to-Speech):** Para la generación de avisos auditivos y lectura de señales.

- **Spacy:** Para el procesamiento de lenguaje natural (NLP) e interpretación de intenciones.

- **MoviePy:** Para la edición de video y sincronización de audio/video.

El sistema combina técnicas avanzadas de visión por computador, procesamiento de lenguaje y síntesis de voz para ofrecer una experiencia accesible y robusta en tiempo real.

## 1. Preparación del Entorno
Instalación de todas las dependencias necesarias para el proyecto.

In [ ]:
# 🔧 Instalación de librerías base (si no están instaladas)
!pip install -q openai-whisper ultralytics git+https://github.com/openai/CLIP.git
!pip install -q moviepy gTTS spacy
!python -m spacy download es_core_news_sm
!pip install -q easyocr

## 2. Configuración y Carga de Modelos
Inicialización de modelos de IA (YOLO, Whisper, Spacy, EasyOCR) y configuración de hardware (GPU/CPU).

In [ ]:
# 1️⃣ CONFIGURACIÓN Y CARGA DE MODELOS (Ejecutar solo una vez)
# ==================================================================================
# Este bloque inicializa todos los modelos de IA y configura el entorno.
# Se ejecuta una sola vez al principio para evitar recargar modelos pesados en cada video.
# ==================================================================================

import os
import torch
import whisper
import spacy
from ultralytics import YOLO
import cv2
import numpy as np
from gtts import gTTS
import easyocr
from IPython.display import Audio, display
import matplotlib.pyplot as plt
import numpy as np
import gc
import time
from spacy.matcher import PhraseMatcher

# Intentar importar MoviePy (Manejo de compatibilidad entre versiones 1.x y 2.x)
try:
    from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip
    IS_MOVIEPY_2 = False
except ImportError:
    from moviepy.video.io.VideoFileClip import VideoFileClip
    from moviepy.audio.io.AudioFileClip import AudioFileClip
    from moviepy.audio.AudioClip import CompositeAudioClip
    IS_MOVIEPY_2 = True

# --- CONFIGURACIÓN DE HARDWARE ---
# Detecta si hay una GPU NVIDIA disponible para acelerar el procesamiento (CUDA)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'✅ Usando dispositivo: {device}')

print("⏳ Cargando modelos (Whisper, Spacy, YOLO, EasyOCR)...")

# --- 1. MODELO DE AUDIO (Whisper) ---
# Se usa para transcribir comandos de voz del usuario a texto.
model_whisper = whisper.load_model('base')

def transcribir_audio(ruta_audio):
    """Convierte un archivo de audio en texto usando OpenAI Whisper."""
    display(Audio(ruta_audio, autoplay=False))
    print(f"🎙️ Transcribiendo {ruta_audio}...")
    resultado = model_whisper.transcribe(ruta_audio, language="es")["text"]
    print(f"🗣️ Usuario dice: '{resultado}'")
    return resultado.strip()


# --- 2. MODELO DE NLP (Spacy) ---
# Se usa para entender la intención del usuario (ej: "¿Es seguro cruzar?" vs "Lee el cartel")
nlp = spacy.load("es_core_news_sm")

# Configuración del PhraseMatcher para detectar intenciones específicas
matcher = PhraseMatcher(nlp.vocab, attr="LEMMA")

# Añadir patrones para diferentes intenciones
matcher.add("VEHICULOS", [nlp("coche"), nlp("vehículo")])
matcher.add("PASO_CEBRA", [nlp("paso de cebra"), nlp("peatón")])
matcher.add("SEMAFORO", [nlp("semáforo"), nlp("luz roja")])
matcher.add("SEÑALES", [nlp("carteles"), nlp("señales")])
matcher.add("SOLO_PELIGRO", [nlp("peligro"), nlp("cuidado"), nlp("alerta"), nlp("silencio")])
matcher.add("TODO", [nlp("todo"), nlp("general")])

# Función para analizar la intención del usuario
def analizar_intencion_spacy(texto):
    doc = nlp(texto.lower())
    matches = matcher(doc)
    # Retorna la primera intención detectada
    if matches:
        return nlp.vocab.strings[matches[0][0]]
    # Si no se detecta ninguna intención, retorna "TODO"
    return "TODO"

# --- 3. MODELO DE VISIÓN (YOLOv8) ---
# Detecta objetos en tiempo real (coches, personas, semáforos, señales).
model_yolo = YOLO('yolov8n.pt')

# --- 4. MODELO DE OCR (EasyOCR) ---
# Lee texto en imágenes. Se configura para español e inglés.
reader = easyocr.Reader(['es', 'en'], gpu=True)
print("✅ Modelos cargados.")

# Clases de interés de COCO dataset para tráfico:
# 0: Persona, 1: Bici, 2: Coche, 3: Moto, 5: Bus, 7: Camión, 9: Semáforo, 11: Señal Stop
CLASES_TRAFICO = [0, 1, 2, 3, 5, 7, 9, 11] 

## ⚙️ Configuración y Carga de Modelos

Este bloque constituye la **fase de inicialización global del sistema SafeStep** y se ejecuta **una única vez** al inicio del entorno. Su objetivo principal es **preparar todos los modelos de inteligencia artificial y dependencias necesarias**, evitando recargas innecesarias que penalizarían gravemente el rendimiento durante el procesamiento de vídeo.

En primer lugar, se importan las librerías fundamentales para **visión artificial, procesamiento de lenguaje, audio y gestión de vídeo**, junto con utilidades auxiliares para visualización, control de memoria y compatibilidad entre versiones. Destaca el manejo explícito de **MoviePy 1.x y 2.x**, garantizando portabilidad del código entre distintos entornos sin romper la ejecución.

A continuación, el sistema detecta automáticamente la disponibilidad de **aceleración por GPU (CUDA)**, permitiendo que los modelos más costosos computacionalmente (YOLO y EasyOCR) se ejecuten en GPU cuando está disponible, o en CPU como alternativa segura. Esta detección dinámica asegura un comportamiento robusto tanto en entornos locales como en notebooks o servidores.

Se inicializan después los **modelos principales del pipeline**:
- **Whisper**, encargado de transcribir comandos de voz del usuario a texto, habilitando una interacción natural y accesible.
- **spaCy**, utilizado para el análisis semántico del texto y la interpretación de la intención del usuario mediante un `PhraseMatcher` basado en lemas, lo que permite reconocer órdenes incluso con variaciones lingüísticas.
- **YOLOv8**, responsable de la detección y seguimiento de objetos críticos del entorno urbano en tiempo real.
- **EasyOCR**, configurado para español e inglés, destinado a la lectura de texto en señales y carteles.

Finalmente, se definen las **clases de tráfico relevantes** del dataset COCO, delimitando explícitamente los objetos que SafeStep considera críticos para la seguridad peatonal (vehículos, peatones, semáforos y señales). Esta selección reduce el ruido visual y optimiza el rendimiento del sistema, alineando la detección con los objetivos funcionales del proyecto.

En conjunto, este bloque establece una **arquitectura modular, eficiente y reutilizable**, sentando las bases técnicas para el procesamiento en tiempo real sin comprometer estabilidad ni escalabilidad.


In [ ]:
# --- FUNCIONES AUXILIARES DE VISIÓN ARTIFICIAL ---

def mejorar_contraste_noche(frame):
    """
    Mejora la visibilidad en condiciones de baja luz usando CLAHE (Contrast Limited Adaptive Histogram Equalization).
    Convierte a espacio de color LAB, mejora el canal de Luminosidad (L) y reconvierte.
    """
    # Convertir a LAB y aplicar CLAHE
    lab = cv2.cvtColor(frame, cv2.COLOR_RGB2LAB)
    # Dividir canales
    l, a, b = cv2.split(lab)
    # Crear objeto CLAHE
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    # Aplicar CLAHE al canal L
    cl = clahe.apply(l)
    # Recombinar canales y convertir de vuelta a RGB
    limg = cv2.merge((cl, a, b))
    return cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)

def analizar_estado_semaforo(img_roi):
    """
    Determina si un semáforo está en ROJO o VERDE analizando la cantidad de píxeles de color en la región detectada.
    Usa rangos de color en espacio HSV.
    """
    # Verificar que la imagen no esté vacía
    if img_roi.size == 0: 
        return "DESCONOCIDO"
    # Convertir a espacio HSV
    hsv = cv2.cvtColor(img_roi, cv2.COLOR_RGB2HSV)
    
    # Rangos para color ROJO (tiene dos rangos en HSV porque el rojo da la vuelta al círculo cromático)
    lower_red1 = np.array([0, 70, 50]); upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 70, 50]); upper_red2 = np.array([180, 255, 255])
    
    # Rango para color VERDE
    lower_green = np.array([35, 70, 50]); upper_green = np.array([90, 255, 255])
    
    # Máscaras para cada color
    mask_r1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask_r2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask_g = cv2.inRange(hsv, lower_green, upper_green)
    
    # Contar píxeles detectados
    red_pixels = cv2.countNonZero(mask_r1) + cv2.countNonZero(mask_r2)
    green_pixels = cv2.countNonZero(mask_g)
    total = img_roi.shape[0] * img_roi.shape[1]
    
    # Umbral del 5% de píxeles para considerar que el color está activo
    if red_pixels > green_pixels and red_pixels > total * 0.05: return "ROJO"
    elif green_pixels > red_pixels and green_pixels > total * 0.05: return "VERDE"
    return "DESCONOCIDO"

def estimar_distancia(y_bottom, h_img):
    """
    Estima la distancia de un objeto basándose en la posición Y de su base en la imagen.
    LIMITACIÓN: Utiliza 'Flat Earth Assumption' (Suposición de Tierra Plana).
    Asume que la cámara está a una altura fija y ángulo constante.
    El horizonte está hardcodeado al 45% de la altura de la imagen.
    """
    # Calcular horizonte
    horizonte = h_img * 0.45 
    if y_bottom <= horizonte: return 99.9
    # 800 es un factor de calibración empírico para la cámara usada
    return round(800 / (y_bottom - horizonte), 1)

def dibujar_paso_cebra_seguro(frame):
    """
    Detecta y resalta pasos de cebra usando umbralización y detección de contornos.
    Busca patrones rectangulares blancos en la parte inferior de la imagen.
    """
    # Convertir a gris y umbralizar
    img_gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    h, w = frame.shape[:2]
    _, mask = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY)
    
    # Solo buscar en la mitad inferior de la imagen
    roi_mask = np.zeros_like(mask); roi_mask[int(h*0.55):, :] = mask[int(h*0.55):, :]
    
    # Detectar contornos
    contours, _ = cv2.findContours(roi_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Dibujar contornos detectados
    overlay = frame.copy(); hay_paso = False
    
    # Filtrar y dibujar contornos que podrían ser franjas del paso de cebra
    for cnt in contours:
        if 200 < cv2.contourArea(cnt) < 50000:
            x, y, w_c, h_c = cv2.boundingRect(cnt)
            # Filtrar por relación de aspecto (las franjas suelen ser rectangulares)
            if w_c / float(h_c) > 0.5:
                cv2.drawContours(overlay, [cnt], -1, (0, 255, 0), -1); hay_paso = True
                
    return cv2.addWeighted(overlay, 0.4, frame, 0.6, 0) if hay_paso else frame

def dibujar_radar(frame, objetos_detectados):
    """
    Dibuja un minimapa tipo radar en la esquina inferior derecha.
    Muestra la posición relativa de los objetos detectados (vista cenital).
    """
    h, w = frame.shape[:2]; radar_size = 150 # Reducido para no ocupar tanta pantalla
    radar_bg = np.zeros((radar_size, radar_size, 3), dtype=np.uint8)
    cx = radar_size // 2; by = radar_size
    
    # Dibujar arcos de distancia (más gruesos y visibles)
    cv2.ellipse(radar_bg, (cx, by), (radar_size, radar_size), 0, 180, 360, (0, 150, 0), 3)
    cv2.circle(radar_bg, (cx, by), int(radar_size * 0.33), (0, 200, 0), 2)
    cv2.circle(radar_bg, (cx, by), int(radar_size * 0.66), (0, 200, 0), 2)
    
    # Dibujar al usuario (punto blanco ajustado)
    cv2.circle(radar_bg, (cx, by - 10), 10, (255, 255, 255), -1)
    
    for (cls, x_c, dist) in objetos_detectados:
        if dist > 30: continue # Solo mostrar objetos cercanos (<30m)
        
        # Mapear coordenadas de pantalla a coordenadas de radar
        r_x = int((x_c / w) * radar_size)
        r_y = max(0, min(radar_size, by - int((dist / 30.0) * radar_size)))
        
        # Rojo para vehículos, Amarillo para otros (Puntos ajustados)
        color = (0, 0, 255) if cls in [2, 3, 5, 7] else (0, 255, 255)
        cv2.circle(radar_bg, (r_x, r_y), 8, color, -1)
        
    # Superponer el radar en el frame original
    roi = frame[h-radar_size-10:h-10, w-radar_size-10:w-10]
    res = cv2.addWeighted(roi, 0.5, radar_bg, 1.0, 0) # Radar totalmente opaco sobre fondo semitransparente
    frame[h-radar_size-10:h-10, w-radar_size-10:w-10] = res
    
    # Marco y etiqueta (Texto ajustado)
    cv2.rectangle(frame, (w-radar_size-10, h-radar_size-10), (w-10, h-10), (255, 255, 255), 4)
    cv2.putText(frame, "RADAR", (w-radar_size, h-radar_size-15), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    return frame

def ocr_carteles_images(img_path, reader, factor=5, mostrar=True):
    # --- Cargar imagen ---
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h_img, w_img = img.shape[:2]

    # --- Detectar zonas claras (cartel) ---
    hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv, (0, 0, 140), (180, 80, 255))
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    candidatos = []

    for cnt in contours:
        x, y, wc, hc = cv2.boundingRect(cnt)
        area = wc * hc
        aspect = wc / max(hc, 1)
        if area > 400 and 1.5 < aspect < 12 and y < h_img * 0.65:
            candidatos.append((x, y, wc, hc, area))

    if not candidatos:
        print("No se detectaron carteles.")
        return img_rgb

    # Ordenar por área
    candidatos.sort(key=lambda c: c[4], reverse=True)

    # Lista de palabras OCR global
    palabras = []

    # --- Procesar cada candidato ---
    for c in candidatos:
        x, y, wc, hc, _ = c
        margin = 8
        x1, y1 = max(0, x - margin), max(0, y - margin)
        x2, y2 = min(w_img, x + wc + margin), min(h_img, y + hc + margin)
        roi = img_rgb[y1:y2, x1:x2]
        roi_big = cv2.resize(roi, None, fx=factor, fy=factor, interpolation=cv2.INTER_CUBIC)

        # OCR
        resultados = reader.readtext(
            roi_big,
            paragraph=False,
            min_size=5,
            text_threshold=0.35,
            low_text=0.25,
            width_ths=0.9
        )

        # Construir lista de palabras
        for bbox, text, prob in resultados:
            escala_x = (x2 - x1) / roi_big.shape[1]
            escala_y = (y2 - y1) / roi_big.shape[0]
            bbox_scaled = [[int(px * escala_x + x1), int(py * escala_y + y1)] for px, py in bbox]
            palabras.append({"text": text, "conf": float(prob), "bbox": bbox_scaled})

    # --- Construir texto final ---
    line_threshold = 25
    lineas = []

    for p in palabras:
        y_center = sum([pt[1] for pt in p["bbox"]]) / 4
        p["y_center"] = y_center
        asignada = False
        for l in lineas:
            if abs(l["y"] - y_center) < line_threshold:
                l["items"].append(p)
                asignada = True
                break
        if not asignada:
            lineas.append({"y": y_center, "items": [p]})

    lineas.sort(key=lambda l: l["y"])
    texto_final = []
    confs = []

    for l in lineas:
        l["items"].sort(key=lambda p: sum([pt[0] for pt in p["bbox"]]) / 4)
        linea_txt = []
        for p in l["items"]:
            palabra_mayus = p["text"].upper()
            linea_txt.append(palabra_mayus)
            confs.append(p["conf"])
        if linea_txt:
            texto_final.append(" ".join(linea_txt))

    texto_final_str = " ".join(texto_final)
    confianza_media = round(np.mean(confs), 2) if confs else 0.0

    # --- Dibujar solo bounding boxes ---
    for p in palabras:
        x_min = min([pt[0] for pt in p["bbox"]])
        y_min = min([pt[1] for pt in p["bbox"]])
        x_max = max([pt[0] for pt in p["bbox"]])
        y_max = max([pt[1] for pt in p["bbox"]])
        cv2.rectangle(img_rgb, (x_min, y_min), (x_max, y_max), (0, 255, 0), 3)

    # --- Dibujar texto final en banda inferior ---
    alto_banda = int(h_img * 0.18)
    overlay = img_rgb.copy()
    cv2.rectangle(overlay, (0, h_img - alto_banda), (w_img, h_img), (0, 0, 0), -1)
    escala = 2.5
    grosor = 4
    (tw, th), _ = cv2.getTextSize(texto_final_str, cv2.FONT_HERSHEY_DUPLEX, escala, grosor)
    while tw > w_img - 20 and escala > 0.5:
        escala -= 0.1
        (tw, th), _ = cv2.getTextSize(texto_final_str, cv2.FONT_HERSHEY_DUPLEX, escala, grosor)
    x_text = max(10, (w_img - tw) // 2)
    y_text = h_img - (alto_banda // 2) + (th // 2)
    cv2.putText(overlay, texto_final_str, (x_text, y_text),
                cv2.FONT_HERSHEY_DUPLEX, escala, (255, 255, 0), grosor, cv2.LINE_AA)

    # --- Imprimir confianzas solo por consola ---
    print("\n--- OCR PALABRA A PALABRA ---")
    for p in palabras:
        print(f"{p['text']:<20} conf={p['conf']:.2f}")
    print("\n--- OCR FINAL ---")
    print(f"Texto: {texto_final_str}")
    print(f"Confianza media: {confianza_media}")

    if mostrar:
        plt.figure(figsize=(12, 6))
        plt.imshow(overlay)
        plt.axis("off")
        plt.show()

    return overlay

# --- GENERACIÓN DE AUDIOS PREGRABADOS ---
# Generamos archivos de audio para las alertas comunes para no usar TTS en tiempo real (latencia).
if not os.path.exists("audios"): 
    os.makedirs("audios")
avisos = {
    "PELIGRO_MOVIMIENTO": "Peligro, vehículo en movimiento detectado. No cruce.",
    "COCHE_QUIETO": "Vehículo detenido detectado.",
    "PEATONES": "Precaución, hay peatones cerca.",
    "SEMAFORO_ROJO": "Semáforo en rojo. Espere.",
    "SEMAFORO_VERDE": "Semáforo en verde. Puede cruzar con precaución.",
    "SEÑAL": "Señal de tráfico detectada."
}
for key, texto in avisos.items():
    f = f"audios/aviso_{key}.mp3"
    if not os.path.exists(f): gTTS(texto, lang='es').save(f)
    
print("✅ Configuración completada. Modelos y audios listos.")

## 🧠 Funciones Auxiliares de Visión Artificial

Este bloque agrupa funciones complementarias destinadas a **mejorar la interpretación visual del entorno**, aportar contexto semántico y reforzar la toma de decisiones del sistema SafeStep de forma eficiente y en tiempo real.

- ### 🌙 Mejora de visibilidad en baja iluminación
Se aplica **CLAHE** sobre el canal de luminosidad (espacio LAB) para aumentar el contraste en escenas nocturnas o con poca luz, mejorando la detección sin amplificar excesivamente el ruido.

- ### 🚦 Análisis del estado del semáforo
Se determina si un semáforo está en **ROJO** o **VERDE** mediante segmentación por color en el espacio **HSV**, contabilizando píxeles relevantes. Es una solución ligera, rápida y suficiente para un elemento crítico de seguridad.

- ### 📏 Estimación de distancia
La distancia a los objetos se estima a partir de la posición vertical de su base en la imagen, utilizando la *Flat Earth Assumption*. Aunque aproximada, esta técnica permite clasificar riesgos cercanos sin necesidad de sensores adicionales.

- ### 🚸 Detección de pasos de cebra
Se identifican patrones blancos rectangulares en la parte inferior de la imagen mediante umbralización y contornos, resaltando visualmente zonas seguras de cruce sin recurrir a modelos pesados.

- ### 🛰️ Radar visual de proximidad
Se muestra un **minimapa tipo radar** que representa la posición relativa y distancia de los objetos detectados desde una vista cenital simplificada, facilitando una comprensión espacial rápida e intuitiva.

- ### 📝 OCR selectivo en imágenes
El reconocimiento de texto se aplica únicamente sobre **imágenes estáticas**, detectando previamente regiones candidatas (carteles) por criterios geométricos y de luminosidad, lo que maximiza la precisión y reduce el coste computacional.

- ### 🔊 Generación de audios pregrabados
Las alertas más comunes se generan previamente mediante TTS y se almacenan como archivos de audio, evitando síntesis en tiempo real y garantizando **baja latencia** en situaciones críticas.

En conjunto, estas funciones actúan como una **capa de apoyo contextual** que mejora la robustez, claridad y eficiencia del sistema sin comprometer el rendimiento en tiempo real.

### ⚠️ Limitación: Señales de Tráfico

El sistema sí detecta **señales de STOP** y les hace Zoom, pero está limitado a esa clase específica del dataset COCO. Si se desea detectar más tipos de señales de tráfico, necesitaríamos un modelo YOLO entrenado específicamente para señales de tráfico (como los de datasets de conducción autónoma). Quizá esto resulte más interesante desde un punto de vista de conductor.

## 3. Definición del Pipeline de Procesamiento
Aquí se define la función principal `procesar_video_safestep` que integra todo el flujo de trabajo: preprocesamiento, detección, OCR, lógica de seguridad y generación de video.

In [ ]:
# 2️⃣ FUNCIÓN DE PROCESAMIENTO (Definición)
def procesar_video_safestep(ruta_video_entrada, ruta_audio, ruta_video_salida=None):
    """
    Procesa un video individual aplicando todo el pipeline de SafeStep.
    Si no se especifica ruta_video_salida, se genera automáticamente.
    """
    if not os.path.exists(ruta_video_entrada):
        print(f"❌ Error: No se encuentra el video {ruta_video_entrada}")
        return None

    # Extraer nombre base del video para usar en archivos temporales
    nombre_base = os.path.splitext(os.path.basename(ruta_video_entrada))[0]

    # --- TRANSCRIPCIÓN DE LA ORDEN DE AUDIO A TEXTO ---
    orden_usuario = transcribir_audio(ruta_audio)
    orden = orden_usuario.lower()

    # --- INTERPRETACIÓN DE LA ORDEN (NLP BÁSICO) ---
    modo_filtrado = analizar_intencion_spacy(orden_usuario)
    print(f"🧠 Orden recibida: '{orden_usuario}' -> Modo: {modo_filtrado}")

    # Generar nombre de salida automático si no se da
    if ruta_video_salida is None:
        # Crear carpeta de salida si no existe
        output_dir = "videos_safestep"
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        # Generar ruta de salida
        nombre = os.path.splitext(os.path.basename(ruta_video_entrada))[0]
        ruta_video_salida = os.path.join(output_dir, f"{nombre}_safestep.mp4")

    print(f"🎬 Procesando video: {ruta_video_entrada} -> {ruta_video_salida}")
    
    # --- CARGA DEL VIDEO Y PARÁMETROS INICIALES ---
    clip = VideoFileClip(ruta_video_entrada)
    fps_original = clip.fps
    w, h = clip.size
    
    # --- OPTIMIZACIÓN DE FPS ---
    # Si el video tiene muchos FPS (>50), procesamos uno de cada dos frames
    # para mantener el rendimiento cercano al tiempo real (~30 FPS).
    saltar_frames = 1
    if fps_original > 50:
        saltar_frames = 2
        print("⚡ Modo Optimizado: Saltando frames para mantener ~30 FPS.")
    
    fps_salida = fps_original / saltar_frames

    # --- GESTIÓN DE MEMORIA EFICIENTE (STREAMING TO DISK) ---
    # En lugar de guardar todos los frames procesados en una lista en RAM (que causaría Memory Leak),
    # escribimos cada frame procesado directamente al disco duro en un archivo temporal.
    # Crear carpeta temporal si no existe para no ensuciar el directorio raíz
    temp_dir = "temp"
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
        
    temp_video_path = os.path.join(temp_dir, f"temp_proc_{nombre_base}.mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_writer = cv2.VideoWriter(temp_video_path, fourcc, fps_salida, (w, h))

    # --- GESTIÓN DE AUDIO INTELIGENTE (PRIORIDADES + COOLDOWNS) ---
    audio_events = [] 
    track_history = {} 
    ultimos_textos_detectados = [] 
    
    # Configuración de prioridades (1 = Máxima, 3 = Mínima)
    PRIORIDADES = {
        "PELIGRO_MOVIMIENTO": 1, "SEMAFORO_ROJO": 1,
        "PEATONES": 2, "SEMAFORO_VERDE": 2,
        "SEÑAL": 3, "COCHE_QUIETO": 3
    }
    
    # Tiempos de espera (segundos) antes de repetir el MISMO aviso
    COOLDOWNS = {
        "PELIGRO_MOVIMIENTO": 8,  # No repetir constantemente
        "SEMAFORO_ROJO": 6,       # Importante recordar si sigue rojo
        "PEATONES": 5,
        "SEMAFORO_VERDE": 8,
        "SEÑAL": 10,
        "COCHE_QUIETO": 15        # Poco relevante, cooldown largo
    }
    
    # Estado del sistema de audio
    last_trigger_time = {k: -999 for k in COOLDOWNS} # Última vez que sonó cada tipo
    global_audio_finish_time = 0 # Momento en que el canal de audio queda libre
    
    """ 
    def trigger_audio_event(event_key, current_time):
        nonlocal global_audio_finish_time
        prioridad = PRIORIDADES.get(event_key, 3)
        cooldown = COOLDOWNS.get(event_key, 10)
        last_time = last_trigger_time.get(event_key, -999)
        
        # Verificar cooldown
        if current_time - last_time < cooldown:
            return
        
        # Verificar si el canal de audio está libre o si la nueva prioridad es mayor
        if current_time >= global_audio_finish_time or prioridad < track_history.get("current_priority", 4):
            audio_path = f"audios/aviso_{event_key}.mp3"
            audio_clip = AudioFileClip(audio_path)
            audio_events.append((current_time, audio_clip))
            global_audio_finish_time = current_time + audio_clip.duration
            last_trigger_time[event_key] = current_time
            track_history["current_priority"] = prioridad 
"""

    # --- BUCLE PRINCIPAL DE PROCESAMIENTO ---
    for i, frame in enumerate(clip.iter_frames()):
        if i % saltar_frames != 0: continue
        
        current_time = i / fps_original
        
        # 1. PREPROCESAMIENTO
        # Mejorar contraste para visión nocturna
        frame_mejorado = mejorar_contraste_noche(frame)
        
        # Dibujar paso de cebra SOLO si es relevante
        if modo_filtrado in ["TODO", "PASO_CEBRA"]:
            img_out = dibujar_paso_cebra_seguro(frame_mejorado)
        else:
            img_out = frame_mejorado.copy()
            
        h_img, w_img = img_out.shape[:2]
        
        # 2. TRACKING DE OBJETOS (YOLOv8)
        # Ejecutamos YOLO primero para saber DÓNDE están los objetos.
        # Usamos el tracker "bytetrack" para mantener la identidad de los objetos entre frames.
        results = model_yolo.track(frame_mejorado, classes=CLASES_TRAFICO, persist=True, verbose=False, conf=0.15, tracker="bytetrack.yaml")
        
        # 3. OCR SELECTIVO (Triggered OCR) - OPTIMIZACIÓN CRÍTICA
        # El OCR es muy lento. En lugar de leer toda la imagen, solo leemos
        # si YOLO detecta una señal de tráfico (Clase 11).
        # Además, solo ejecutamos OCR cada 15 frames para ahorrar recursos.
        if modo_filtrado in ["TODO", "SEÑALES"] and i % 30 == 0:
            ultimos_textos_detectados = []

            # Convertir frame a RGB si no lo está
            frame_rgb = cv2.cvtColor(frame_mejorado, cv2.COLOR_BGR2RGB)
            h_img, w_img = frame_rgb.shape[:2]

            # --- Detectar zonas claras similares a carteles ---
            hsv = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2HSV)
            mask = cv2.inRange(hsv, (0, 0, 140), (180, 80, 255))
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 8))
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            for cnt in contours:
                x, y, wc, hc = cv2.boundingRect(cnt)
                area = wc * hc
                aspect = wc / max(hc, 1)

                # Filtrar zonas demasiado pequeñas o proporciones extrañas
                if area < 400 or aspect < 1.5 or aspect > 12 or y > h_img * 0.65:
                    continue

                margin = 8
                x1, y1 = max(0, x - margin), max(0, y - margin)
                x2, y2 = min(w_img, x + wc + margin), min(h_img, y + hc + margin)
                roi = frame_rgb[y1:y2, x1:x2]
                roi_big = cv2.resize(roi, None, fx=5, fy=5, interpolation=cv2.INTER_CUBIC)

                try:
                    lecturas = reader.readtext(roi_big, paragraph=False, min_size=5,
                                            text_threshold=0.35, low_text=0.25, width_ths=0.9)
                    for bbox, text, prob in lecturas:
                        if prob < 0.4:  # ignorar confianza baja
                            continue
                        # Reescalar bbox al tamaño del frame original
                        escala_x = (x2 - x1) / roi_big.shape[1]
                        escala_y = (y2 - y1) / roi_big.shape[0]
                        bbox_scaled = [[int(px * escala_x + x1), int(py * escala_y + y1)] for px, py in bbox]
                        ultimos_textos_detectados.append((bbox_scaled, text, prob))
                except Exception as e:
                    print(f"Error OCR en ROI: {e}")

        # Dibujar textos detectados (Persistencia visual para que no parpadeen)
        # SOLO si el modo incluye lectura de señales
        if modo_filtrado in ["TODO", "SEÑALES"]:
            for (bbox, text, prob) in ultimos_textos_detectados:
                (tl, tr, br, bl) = bbox
                x1, y1 = int(tl[0]), int(tl[1]); x2, y2 = int(br[0]), int(br[1])
                cv2.rectangle(img_out, (x1, y1), (x2, y2), (255, 100, 0), 2)
                cv2.putText(img_out, text.upper(), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.8, (255, 100, 0), 4)
                
                # Efecto Lupa: Mostrar el texto ampliado en una esquina
                if (x2 - x1) < 100: 
                    try:
                        roi = frame_mejorado[y1:y2, x1:x2]
                        zoom_size = 200
                        roi_zoom = cv2.resize(roi, (zoom_size, zoom_size))
                        roi_zoom = cv2.copyMakeBorder(roi_zoom, 5, 5, 5, 5, cv2.BORDER_CONSTANT, value=(255, 100, 0))
                        y_off, x_off = h_img - zoom_size - 20, w_img - zoom_size - 20
                        img_out[y_off:y_off+zoom_size+10, x_off:x_off+zoom_size+10] = roi_zoom
                        cv2.putText(img_out, "TEXTO", (x_off, y_off - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 100, 0), 2)
                    except: pass

        # 4. LÓGICA DE SEGURIDAD Y VISUALIZACIÓN
        frame_alerts = [] # Lista de alertas detectadas en ESTE frame
        mensaje_visual = ""
        color_mensaje = (255, 255, 255)
        peligro_inminente = False
        objetos_radar = [] 

        if results and results[0]:
            boxes = results[0].boxes
            if boxes.id is not None:
                track_ids = boxes.id.int().cpu().tolist()
                cls_ids = boxes.cls.int().cpu().tolist()
                coords = boxes.xyxy.cpu().numpy()
                
                for box_id, cls, xyxy in zip(track_ids, cls_ids, coords):
                    x1, y1, x2, y2 = map(int, xyxy)
                    center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
                    
                    # Estimar distancia
                    distancia_m = estimar_distancia(y2, h_img)
                    texto_dist = f"{distancia_m}m"
                    
                    # --- LÓGICA DE MOVIMIENTO (Siempre calcular para mantener estado) ---
                    es_movimiento = False
                    if cls in [2, 3, 5, 7]: 
                        if box_id in track_history:
                            prev_x, prev_y = track_history[box_id]
                            desplazamiento = np.sqrt((center_x - prev_x)**2 + (center_y - prev_y)**2)
                            if desplazamiento > (5.0 * saltar_frames): es_movimiento = True
                        track_history[box_id] = (center_x, center_y)

                    # --- FILTRADO VISUAL: ¿Debemos mostrar este objeto? ---
                    mostrar_visual = False
                    if modo_filtrado == "TODO": mostrar_visual = True
                    elif modo_filtrado == "VEHICULOS" and cls in [2, 3, 5, 7]: mostrar_visual = True
                    elif modo_filtrado == "PASO_CEBRA" and cls == 0: mostrar_visual = True # Peatones relevantes para cruce
                    elif modo_filtrado == "SEMAFORO" and cls == 9: mostrar_visual = True
                    elif modo_filtrado == "SEÑALES" and cls == 11: mostrar_visual = True
                    elif modo_filtrado == "SOLO_PELIGRO" and es_movimiento: mostrar_visual = True
                    
                    if not mostrar_visual: 
                        continue

                    # Si pasa el filtro, lo añadimos al radar
                    objetos_radar.append((int(cls), center_x, distancia_m))
                    
                    # --- LÓGICA PARA VEHÍCULOS ---
                    if cls in [2, 3, 5, 7]: 
                        color_box = (200, 0, 0) if es_movimiento else (0, 0, 255)
                        cv2.rectangle(img_out, (x1, y1), (x2, y2), color_box, 8)
                        cv2.putText(img_out, texto_dist, (x1, y1 - 35), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 0), 4)
                        
                        #-- LÓGICA DE ALERTAS PARA VEHÍCULOS ---
                        if es_movimiento:
                            frame_alerts.append("PELIGRO_MOVIMIENTO")
                            mensaje_visual = f"PELIGRO: VEHICULO A {distancia_m}m"
                            color_mensaje = (255, 0, 0); peligro_inminente = True
                        else: frame_alerts.append("COCHE_QUIETO")
                            
                    # --- LÓGICA PARA PEATONES ---
                    elif cls == 0: 
                        if (y2 - y1) > h_img * 0.3: # Solo si está cerca/grande
                            frame_alerts.append("PEATONES")
                            if "PELIGRO_MOVIMIENTO" not in frame_alerts:
                                mensaje_visual = "CUIDADO: PEATONES"
                                color_mensaje = (255, 255, 0)
                        # Amarillo (255, 255, 0) para alta visibilidad y precaución
                        cv2.rectangle(img_out, (x1, y1), (x2, y2), (255, 255, 0), 8)

                    # --- LÓGICA PARA SEMÁFOROS Y SEÑALES ---
                    elif cls in [9, 11]:
                        roi = frame_mejorado[y1:y2, x1:x2]
                        color_borde = (255, 165, 0)
                        if cls == 9: # Semáforo
                            estado_sem = analizar_estado_semaforo(roi)
                            if estado_sem == "ROJO":
                                frame_alerts.append("SEMAFORO_ROJO")
                                mensaje_visual = "SEMAFORO ROJO: ESPERE"
                                color_mensaje = (255, 0, 0); color_borde = (255, 0, 0)
                            elif estado_sem == "VERDE":
                                frame_alerts.append("SEMAFORO_VERDE")
                                if "PELIGRO_MOVIMIENTO" not in frame_alerts and "SEMAFORO_ROJO" not in frame_alerts:
                                    mensaje_visual = "VERDE: PUEDE CRUZAR"
                                    color_mensaje = (0, 255, 0); color_borde = (0, 255, 0)
                        
                        # Dibujar Lupa para ver mejor la señal/semáforo
                        try: 
                            zoom_size = 200
                            roi_zoom = cv2.resize(roi, (zoom_size, zoom_size))
                            roi_zoom = cv2.copyMakeBorder(roi_zoom, 5, 5, 5, 5, cv2.BORDER_CONSTANT, value=color_borde)
                            y_off, x_off = 20, w_img - zoom_size - 20
                            if y_off + zoom_size + 10 < h_img and x_off + zoom_size + 10 < w_img:
                                img_out[y_off:y_off+zoom_size+10, x_off:x_off+zoom_size+10] = roi_zoom
                                cv2.putText(img_out, "ZOOM", (x_off, y_off - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color_borde, 2)
                        except: pass
                        cv2.rectangle(img_out, (x1, y1), (x2, y2), color_borde, 8)

        # 5. RADAR Y FEEDBACK FINAL
        img_out = dibujar_radar(img_out, objetos_radar)
        
        # Alerta visual de pantalla completa si hay peligro inminente
        # SOLO si estamos en un modo que muestre peligros o TODO
        if peligro_inminente and modo_filtrado in ["TODO", "VEHICULOS", "SOLO_PELIGRO"]:
             cv2.rectangle(img_out, (0, 0), (w_img, h_img), (0, 0, 255), 10)

        # --- PROCESADOR DE AUDIO CENTRALIZADO ---
        aviso_final = None
        
        # 0. Filtrado por Intención del Usuario (NLP)
        alertas_filtradas = []
        for a in frame_alerts:
            # Siempre permitimos PELIGRO_MOVIMIENTO por seguridad crítica
            if a == "PELIGRO_MOVIMIENTO": 
                alertas_filtradas.append(a)
                continue
                
            if modo_filtrado == "TODO":
                alertas_filtradas.append(a)
            elif modo_filtrado == "SEMAFORO" and "SEMAFORO" in a:
                alertas_filtradas.append(a)
            elif modo_filtrado == "VEHICULOS" and ("COCHE" in a or "MOVIMIENTO" in a):
                alertas_filtradas.append(a)
            elif modo_filtrado == "PASO_CEBRA" and "PEATONES" in a: # Peatones suelen implicar cruce
                alertas_filtradas.append(a)
            elif modo_filtrado == "SEÑALES" and "SEÑAL" in a:
                alertas_filtradas.append(a)
            elif modo_filtrado == "SOLO_PELIGRO":
                pass # Ya se añadió PELIGRO_MOVIMIENTO arriba
        
        # 1. Filtrar alertas por Cooldown (¿Hace cuánto sonó esta misma alerta?)
        alertas_disponibles = [a for a in alertas_filtradas if (current_time - last_trigger_time.get(a, -999)) > COOLDOWNS.get(a, 5)]
        
        # 2. Ordenar por Prioridad (Menor número = Mayor importancia)
        if alertas_disponibles:
            alertas_disponibles.sort(key=lambda x: PRIORIDADES.get(x, 99))
            mejor_alerta = alertas_disponibles[0] # La más importante
            
            # 3. Chequear disponibilidad del canal de audio (¿Ha terminado el anterior?)
            # Permitimos solapamiento leve (0.5s) para alertas de Prioridad 1
            margen = 0.5 if PRIORIDADES.get(mejor_alerta, 99) == 1 else 0
            
            if current_time >= (global_audio_finish_time - margen):
                aviso_final = mejor_alerta
                
                # Registrar evento
                audio_file = f"audios/aviso_{aviso_final}.mp3"
                if os.path.exists(audio_file):
                    audio_events.append((current_time, audio_file))
                    
                    # Actualizar tiempos
                    last_trigger_time[aviso_final] = current_time
                    # Estimamos duración del audio en 2.5s (promedio)
                    global_audio_finish_time = current_time + 2.5 
                    print(f"🔊 {aviso_final} en {current_time:.2f}s (Prioridad: {PRIORIDADES.get(aviso_final)})")

        # Dibujar mensaje de texto inferior (Aumentado para baja visión)
        if mensaje_visual:
            # Escala reducida para no ocupar toda la pantalla
            font_scale = 1.2
            thickness = 3
            (text_w, text_h), baseline = cv2.getTextSize(mensaje_visual, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)
            
            # Fondo negro de alto contraste más grande
            cv2.rectangle(img_out, (10, h_img - text_h - 40), (10 + text_w + 40, h_img - 10), (0, 0, 0), -1)
            # Borde blanco grueso para resaltar sobre fondo oscuro
            cv2.rectangle(img_out, (10, h_img - text_h - 40), (10 + text_w + 40, h_img - 10), (255, 255, 255), 4)
            
            cv2.putText(img_out, mensaje_visual, (30, h_img - 25), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color_mensaje, thickness)

        # ESCRIBIR FRAME A DISCO (Evitar Memory Leak)
        # Convertir RGB (MoviePy) a BGR (OpenCV) antes de escribir
        out_writer.write(cv2.cvtColor(img_out, cv2.COLOR_RGB2BGR))
        
        if i % 10 == 0: print(f"Procesado frame {i}...", end="\r")

    out_writer.release()
    print("\n💾 Generando video final con audio...")
    
    # --- POST-PROCESADO DE AUDIO ---
    # Cargar el video mudo procesado y añadirle los eventos de audio
    clip_video = VideoFileClip(temp_video_path)
    
    if audio_events:
        audioclips = []
        for t, audio_file in audio_events:
            try:
                aclip = AudioFileClip(audio_file)
                aclip = aclip.with_start(t) if IS_MOVIEPY_2 else aclip.set_start(t)
                audioclips.append(aclip)
            except: pass
        if audioclips:
            final_audio = CompositeAudioClip(audioclips)
            final_audio = final_audio.with_duration(clip_video.duration) if IS_MOVIEPY_2 else final_audio.set_duration(clip_video.duration)
            clip_video = clip_video.with_audio(final_audio) if IS_MOVIEPY_2 else clip_video.set_audio(final_audio)

    clip_video.write_videofile(ruta_video_salida, codec="libx264", audio_codec="aac")
    
   # --- CIERRE EXPLÍCITO DE RECURSOS (CRÍTICO EN WINDOWS) ---
    try:
        clip_video.close()
    except:
        pass
    try:
        clip.close()  # cerrar el video de entrada
    except:
        pass
    # Forzar liberación de memoria y handles
    gc.collect()
    time.sleep(0.5)
    # --- LIMPIEZA DE ARCHIVOS TEMPORALES ---
    if os.path.exists(temp_video_path):
        try:
            os.remove(temp_video_path)
        except PermissionError:
            print("⚠️ No se pudo borrar el archivo temporal (Windows lo mantiene abierto)")
    # Intentar borrar carpeta temp si está vacía
    try:
        os.rmdir(temp_dir)
    except:
        pass
            
    print(f"✅ Video guardado: {ruta_video_salida}")
    return ruta_video_salida

## 🎬 Función de Procesamiento Principal — `procesar_video_safestep`

Esta función implementa el **pipeline completo de SafeStep** para procesar un vídeo de forma eficiente, segura y adaptada a la intención del usuario.

### 🎙️ Entrada multimodal
- Transcribe comandos de voz con **Whisper**.
- Interpreta la intención mediante **NLP (SpaCy)** para activar modos selectivos (vehículos, señales, semáforos, etc.).

### ⚡ Procesamiento optimizado
- Ajuste dinámico de FPS para mantener rendimiento cercano al tiempo real.
- Escritura directa a disco (*streaming to disk*) para evitar consumo excesivo de memoria.

### 👁️ Visión artificial en tiempo real
- **YOLOv8 + ByteTrack** para detección y seguimiento de objetos.
- Mejora de contraste nocturno y detección opcional de pasos de cebra.
- **OCR selectivo** ejecutado cada N frames y solo sobre los vídeos cuya intención del usuario sea carteles/señales (SEÑALES) y cualquier alerta (TODO).

### 🚨 Lógica de seguridad
- Estimación de distancia y detección de movimiento.
- Priorización de alertas con sistema de **prioridades y cooldowns**.
- Mensajes visuales de alto contraste y alertas de peligro inminente.

### 🛰️ Feedback visual y auditivo
- Radar de proximidad para contexto espacial.
- Reproducción de audios pregrabados sincronizados con el vídeo final.

### 🧹 Postprocesado robusto
- Composición final de vídeo + audio.
- Liberación explícita de recursos y limpieza de archivos temporales.

➡️ Resultado: un vídeo final anotado, accesible y optimizado, listo para asistir al usuario en entornos urbanos reales.


## 4. Ejecución y Pruebas
Bloque para ejecutar el pipeline sobre un video de prueba y verificar los resultados.

### 4.1. Vehículos en Movimiento

In [ ]:
# 3️⃣ EJECUCIÓN MANUAL - VIDEO 1 (Tráfico)
# Escenario: Usuario esperando en una parada o borde de acera.
ruta_audio1 = "audios_ordenes/orden1.mp3"

from IPython.display import Video, display
video_a_procesar = "videos/video1.mp4"

if os.path.exists(video_a_procesar):
    ruta_salida = procesar_video_safestep(video_a_procesar, ruta_audio1)
    if ruta_salida and os.path.exists(ruta_salida):
        display(Video(ruta_salida, embed=True, html_attributes="controls width=640"))
else:
    print(f"⚠️ No se encuentra el archivo: {video_a_procesar}")

### 4.2. Pasos de cebra

In [ ]:
# 3️⃣ EJECUCIÓN MANUAL - VIDEO 2 (Navegación)
# Escenario: Usuario buscando cruzar la calle.
ruta_audio2 = "audios_ordenes/orden2.mp3"

from IPython.display import Video, display
video_a_procesar = "videos/video2.mp4"

if os.path.exists(video_a_procesar):
    ruta_salida = procesar_video_safestep(video_a_procesar, ruta_audio2)
    if ruta_salida and os.path.exists(ruta_salida):
        display(Video(ruta_salida, embed=True, html_attributes="controls width=640"))
else:
    print(f"⚠️ No se encuentra el archivo: {video_a_procesar}")

### ⚠️ Desafío: Falsos Positivos en la Detección de Pasos de Cebra

- **Problema:** Aunque la función detecta correctamente muchos pasos de cebra reales, en ocasiones puede identificar como paso de cebra otros elementos del entorno, como franjas en el pavimento, marcas viales, sombras o incluso zonas de acera con patrones similares.

- **Causa técnica:** La función `analizar_paso_cebra` se basa en la detección de líneas blancas paralelas y su disposición geométrica. Sin embargo, no incorpora un análisis semántico profundo ni contexto urbano, por lo que cualquier patrón visual que cumpla con los criterios geométricos y de color puede ser interpretado erróneamente como paso de cebra.

- **Ejemplo:** Unas losas blancas alineadas en una acera, o marcas de aparcamiento, pueden ser confundidas con pasos de cebra si su forma y color se parecen lo suficiente.

---

### 📝 Justificación y Posibles Mejoras

- **Justificación:** El método empleado es eficiente y rápido, pero limitado por su simplicidad. Es adecuado para prototipos y entornos controlados, pero puede fallar en escenarios urbanos complejos.

- **Mejoras posibles:** Para reducir falsos positivos, se podrían emplear modelos de aprendizaje profundo entrenados específicamente para pasos de cebra, o combinar la detección geométrica con información contextual (por ejemplo, la presencia de semáforos, cruces o señales verticales).

### 4.3. Semáforos

In [ ]:
# 3️⃣ EJECUCIÓN MANUAL - VIDEO 3 (Semáforos)
# Escenario: Usuario en un cruce regulado, quiere saber cuándo cruzar.
ruta_audio3 = "audios_ordenes/orden3.mp3"

from IPython.display import Video, display
video_a_procesar = "videos/video3.mp4"

if os.path.exists(video_a_procesar):
    ruta_salida = procesar_video_safestep(video_a_procesar, ruta_audio3)
    if ruta_salida and os.path.exists(ruta_salida):
        display(Video(ruta_salida, embed=True, html_attributes="controls width=640"))
else:
    print(f"⚠️ No se encuentra el archivo: {video_a_procesar}")

# 3️⃣ EJECUCIÓN MANUAL - VIDEO 4 (Semáforos)
video_a_procesar = "videos/video4.mp4"

if os.path.exists(video_a_procesar):
    ruta_salida = procesar_video_safestep(video_a_procesar, ruta_audio3)
    if ruta_salida and os.path.exists(ruta_salida):
        display(Video(ruta_salida, embed=True, html_attributes="controls width=640"))
else:
    print(f"⚠️ No se encuentra el archivo: {video_a_procesar}")

### ⚠️ Análisis de Caso de Borde: Falso Positivo en Video 4

Durante la ejecución del `video4.mp4`, se puede observar una **detección errónea de semáforo**.

**Justificación Técnica:**
El sistema identifica correctamente una señal de **STOP**, pero el algoritmo de análisis de estado (`analizar_estado_semaforo`) confunde la gran cantidad de píxeles rojos de la señal de STOP con la luz roja de un semáforo. Esto ocurre porque la lógica de detección de color busca picos de rojo en la región de interés, y al ser la señal de STOP predominantemente roja, genera un falso positivo de "Semáforo en Rojo".

El `video3.mp4` se incluye como contraprueba para demostrar el funcionamiento correcto del algoritmo en un escenario donde no existe esta ambigüedad visual, validando la lógica de detección de semáforos en condiciones normales.

### 4.4. Carteles

In [ ]:
# 3️⃣ EJECUCIÓN MANUAL - VIDEO 5 (Carteles)
# Escenario: Usuario en entorno desconocido, quiere indicación de carteles.
ruta_audio4 = "audios_ordenes/orden4.mp3"

from IPython.display import Video, display
video_a_procesar = "videos/video5.mp4"

if os.path.exists(video_a_procesar):
    ruta_salida = procesar_video_safestep(video_a_procesar, ruta_audio4)
    if ruta_salida and os.path.exists(ruta_salida):
        display(Video(ruta_salida, embed=True, html_attributes="controls width=640"))
else:
    print(f"⚠️ No se encuentra el archivo: {video_a_procesar}")

### ❓ Por qué el OCR de carteles en vídeo puede fallar

En la ejecución sobre el **video 5 (carteles)**, el sistema no logra detectar en su totalidad el texto del cartel. Esto puede deberse a varias causas técnicas habituales en visión artificial:

- **Baja resolución del video:** Los carteles aparecen demasiado pequeños para que el OCR pueda distinguir los caracteres.
- **Desenfoque o movimiento:** El vídeo puede estar borroso, dificultando la segmentación y lectura.
- **Iluminación deficiente o reflejos:** Sombras, contraluces o brillos pueden ocultar parte del texto.
- **Compresión y artefactos:** Los vídeos suelen tener artefactos de compresión que degradan la calidad de los bordes y el contraste.
- **Ángulo o perspectiva:** Si el cartel está inclinado o deformado por la perspectiva, el OCR puede fallar.
- **Ruido visual:** Elementos del fondo, otros objetos o textos cercanos pueden confundir al sistema.

Además, al hacer uso de OCR cada 30 frames (aprox. cada 1 segundos) en toda la imagen, el procesamiento es muy lento. Se han probado diferentes técnicas para su corrección, como analizar solo el 40% de la imagen superior, donde suelen estar los carteles, pero esta decisión representaba una limitación enorme en cuanto a la detección de textos útiles para el ciclo peatonal. Por ende, se ha de considerar su optimización para versiones más avanzadas del proyecto actual.

#### 🖼️ OCR en imágenes fijas: una alternativa robusta

Para demostrar la utilidad y precisión del OCR en el contexto de SafeStep Peatonal, se ha añadido una herramienta específica para **detectar y leer carteles en imágenes fijas**. Esta técnica permite:

- Probar el OCR sobre capturas de pantalla o fotos de carteles tomadas con el móvil.
- Ajustar el preprocesamiento y la ampliación para maximizar la legibilidad.
- Visualizar el resultado con un diseño accesible para personas con baja visión.

**Conclusión:**  
Aunque el OCR en vídeo es un reto por las limitaciones técnicas mencionadas, el reconocimiento de texto en imágenes fijas funciona de forma fiable y puede integrarse en flujos de ayuda peatonal, por ejemplo, permitiendo al usuario tomar una foto de un cartel y recibir la lectura accesible del texto.

In [ ]:
# Prueba OCR en una imagen fija
rutas = [
    "images/cartel1.jpg",
    "images/cartel2.jpg",
    "images/cartel3.png",
    "images/cartel4.png"
]

for ruta in rutas:
    if os.path.exists(ruta):
        print(f"\n🔍 Probando OCR en imagen: {ruta}")
        img_resultado = ocr_carteles_images(ruta, reader, mostrar=True)
    else:
        print(f"⚠️ No se encuentra la imagen: {ruta}")

### 4.5. Alertas Críticas

In [ ]:
# 3️⃣ EJECUCIÓN MANUAL - VIDEO 6 (Modo Silencioso)
# Escenario: Usuario en entorno ruidoso o conocido, solo quiere alertas críticas.
ruta_audio5 = "audios_ordenes/orden5.mp3"

from IPython.display import Video, display
video_a_procesar = "videos/video6.mp4"

if os.path.exists(video_a_procesar):
    ruta_salida = procesar_video_safestep(video_a_procesar, ruta_audio5)
    if ruta_salida and os.path.exists(ruta_salida):
        display(Video(ruta_salida, embed=True, html_attributes="controls width=640"))
else:
    print(f"⚠️ No se encuentra el archivo: {video_a_procesar}")

### ⚠️ Desafío: Distinguir Movimiento Real de Objetos vs. Movimiento de Cámara

- **Problema:** Si la cámara no es estática (por ejemplo, si el usuario camina o mueve el móvil), los objetos pueden parecer que se mueven aunque estén quietos.
- **Solución profesional:** La mejor práctica sería compensar el movimiento global de la escena usando técnicas como Optical Flow o Keypoints para estimar el desplazamiento de fondo y restarlo al movimiento de cada objeto. Así, solo se detecta movimiento real relativo al entorno.

### 4.6. General

In [ ]:
# 3️⃣ EJECUCIÓN MANUAL - VIDEO 7 (General)
# Escenario: Usuario caminando por calle normal. Quiere saber todo.
ruta_audio6 = "audios_ordenes/orden6.mp3"

from IPython.display import Video, display
video_a_procesar = "videos/video7.mp4"

if os.path.exists(video_a_procesar):
    ruta_salida = procesar_video_safestep(video_a_procesar, ruta_audio6)
    if ruta_salida and os.path.exists(ruta_salida):
        display(Video(ruta_salida, embed=True, html_attributes="controls width=640"))
else:
    print(f"⚠️ No se encuentra el archivo: {video_a_procesar}")

# 3️⃣ EJECUCIÓN MANUAL - VIDEO 8 (General)
video_a_procesar = "videos/video8.mp4"

if os.path.exists(video_a_procesar):
    ruta_salida = procesar_video_safestep(video_a_procesar, ruta_audio6)
    if ruta_salida and os.path.exists(ruta_salida):
        display(Video(ruta_salida, embed=True, html_attributes="controls width=640"))
else:
    print(f"⚠️ No se encuentra el archivo: {video_a_procesar}")

### ⚠️ Desafío: Sobrecarga de Información y Eficiencia

- **Problema:** En entornos urbanos muy estimulados (muchos coches, peatones, señales, semáforos, etc.), tratar de identificar y avisar de todo puede generar una sobrecarga de información y de avisos de audio.
- **Consecuencia:** El usuario puede recibir demasiados avisos, lo que puede resultar confuso, molesto o incluso ineficiente para la toma de decisiones seguras.
- **Reflexión:** Este es uno de los grandes retos de los sistemas de asistencia multimodal: encontrar el equilibrio entre informar y no saturar al usuario. En la práctica, sería necesario priorizar, filtrar y adaptar los avisos según el contexto y las necesidades del usuario.

## 5. Justificaciones Técnicas y Decisiones de Diseño

Este documento recoge las decisiones arquitectónicas clave implementadas en **SafeStep** para equilibrar precisión, rendimiento y viabilidad técnica.

---

### 1. ⚡ Optimización de FPS (Frame Skipping)
> **Problema:** El procesamiento de video en alta resolución (1080p/60fps) satura la capacidad de procesamiento en tiempo real.  
> **Solución:** Implementación de un algoritmo de salto de frames dinámico.
*   **Mecanismo:** Si el video de entrada supera los 50 FPS, el sistema procesa solo 1 de cada 2 frames.
*   **Impacto:** Mantiene una tasa de salida estable de ~30 FPS, suficiente para la percepción humana, reduciendo la carga computacional a la mitad sin perder información crítica.

### 2. 🎯 OCR Híbrido
> **Problema:** El reconocimiento óptico de caracteres (OCR) es computacionalmente costoso y propenso a leer "ruido" (texto en camisetas, carteles irrelevantes).  
> **Solución:** Solo se ejecuta cuando la intención del usuario es SEÑALES, enfocándose en carteles relevantes:
*   **Modo SEÑALES o TODO:** Escanea regiones del frame cada 30 frames para detectar texto de señalización urbana. No depende de YOLO.
*   **Otros Modos:** El OCR no se ejecuta, evitando gasto computacional innecesario.
*   **Impacto:** Minimiza carga de procesamiento y reduce falsos positivos, garantizando lectura de carteles solo cuando el usuario lo requiere..

### 3. 💾 Gestión de Memoria (Streaming to Disk)
> **Problema:** Almacenar frames procesados en listas de Python (`frames.append()`) provoca desbordamiento de memoria (RAM) en videos largos (>1 min), causando cierres inesperados.  
> **Solución:** Escritura en flujo continuo al disco.
*   **Mecanismo:** Uso de `cv2.VideoWriter` para escribir cada frame procesado inmediatamente al almacenamiento secundario y liberar la memoria RAM al instante.
*   **Impacto:** Consumo de RAM **constante y bajo** (O(1)), permitiendo procesar videos de duración indefinida sin colapsar el sistema.

### 4. 📏 Estimación de Distancia "Flat Earth"
> **Problema:** Calcular la distancia real (profundidad) con una sola cámara (monocular) es un problema mal planteado sin referencias externas.  
> **Solución:** Aproximación geométrica basada en la suposición de "Tierra Plana".
*   **Mecanismo:** Se asume que la cámara está a una altura fija y el suelo es plano. La distancia se estima inversamente proporcional a la posición `y` de la base del objeto en la imagen (`d ~ 1 / (y - horizonte)`).
*   **Impacto:** Cálculo instantáneo con error aceptable para alertas de seguridad (cerca/lejos), sin necesidad de sensores LiDAR o cámaras estéreo costosas.

### 5. 🧩 Arquitectura Modular
> **Problema:** La carga de modelos pesados (Whisper, YOLO, Spacy) tarda varios segundos.  
> **Solución:** Separación del ciclo de vida.
*   **Mecanismo:** La inicialización de modelos se realiza en una celda independiente ("Configuración") que se ejecuta una sola vez. La función de procesamiento reutiliza estos objetos en memoria.
*   **Impacto:** Permite iterar y procesar múltiples videos rápidamente sin tiempos de espera de carga repetitivos.

### 6. 🗣️ Integración de Comandos de Voz y Procesamiento de Lenguaje Natural
> **Problema:** La interacción por texto limita la accesibilidad y naturalidad del sistema.
> **Solución:** Uso de Whisper para transcribir órdenes de voz y spaCy para interpretar la intención del usuario.
*   **Mecanismo:** El usuario puede grabar una orden hablada, que se transcribe a texto con Whisper y se analiza con spaCy para determinar el modo de funcionamiento.
*   **Impacto:** Permite interacción multimodal, mayor accesibilidad y flexibilidad en la configuración del sistema.

### 7. 🎧 Gestión de Audio: Prioridades y Cooldowns
> **Problema:** La "fatiga de alertas" ocurre cuando el sistema repite el mismo mensaje constantemente (ej: "Coche detectado") impidiendo escuchar otros avisos importantes.
> **Solución:** Sistema de cola de audio inteligente.
*   **Prioridades:** Se asigna un nivel de importancia a cada evento (1: Peligro Inminente, 2: Precaución, 3: Info). Si ocurren dos eventos a la vez, solo suena el más crítico.
*   **Cooldowns Individuales:** Cada tipo de alerta tiene su propio temporizador. Si suena "Coche", ese mensaje específico se silencia por 10s, pero el canal queda libre para "Semáforo Rojo" inmediatamente.
*   **Impacto:** Reduce el estrés del usuario y garantiza que la información nueva y crítica siempre tenga preferencia sobre la repetición de información conocida.

### 8. 👁️ Accesibilidad Visual Mejorada
> **Problema:** Los elementos gráficos estándar (bounding boxes de 2px, textos pequeños) son difíciles de percibir para usuarios con baja visión.
> **Solución:** Escalado agresivo de elementos de interfaz.
*   **Mecanismo:** Aumento del grosor de líneas a 8-10px, tamaño de fuente a escala 3.0 y radar ampliado a 400px.
*   **Impacto:** Garantiza que las alertas visuales sean perceptibles incluso con agudeza visual reducida, priorizando la legibilidad sobre la estética convencional.

### 9. 📡 Radar de Seguridad Permanente
> **Problema:** Al filtrar por modo (ej: "Solo semáforos"), el usuario pierde consciencia situacional de vehículos cercanos que podrían ser peligrosos.
> **Solución:** Radar siempre activo para vehículos.
*   **Mecanismo:** Los vehículos (coches, motos, buses, camiones) **siempre** aparecen en el radar independientemente del modo seleccionado. El resto de objetos solo aparecen si son relevantes para el modo actual.
*   **Impacto:** Mantiene la seguridad crítica del usuario sin saturar la visualización principal, permitiendo foco en la tarea solicitada sin perder alertas de proximidad vehicular.

### 10. 🧠 Consideraciones Éticas y de Seguridad
> **Limitación:** SafeStep no sustituye la percepción humana ni garantiza decisiones seguras por sí solo. Es decir, el sistema es asistivo, por lo que el usuario, como se ha visto, no debe depositar confianza plena en el buen funcionamiento del SafeStep, ya que puede haber fallos de percepción, latencias o errores de clasificación.

### 11. 🌍 Dependencia del contexto urbano
> **Problema:** A pesar de que el SafeStep cubre la mayoría de los contextos y generaliza bastante bien, puede ser que en entornos urbanos externos la normativa visual sea distinta y se haga uso de una señalización no convencional, de manera que el sistema no funcionaría con tanto éxito como en zonas/países estándar.

## 6. 📊 Evaluación final del sistema


---

- **SafeStep** confirma el enorme potencial de los sistemas de asistencia peatonal basados en **Visión Artificial, Realidad Aumentada y procesamiento multimodal** como herramientas de apoyo real para personas con visión reducida. El sistema logra integrar de forma eficaz detección de objetos, OCR, análisis de movimiento, estimación de distancia, comprensión del lenguaje natural y generación de alertas visuales y sonoras, manteniendo un equilibrio sólido entre **seguridad, accesibilidad y rendimiento en tiempo real**.

- Las decisiones de diseño adoptadas a lo largo del proyecto —como el **OCR selectivo**, el **filtrado por intención del usuario**, la **gestión inteligente de prioridades y cooldowns de audio**, y la **optimización mediante salto de frames**— han resultado fundamentales para evitar tanto la sobrecarga computacional como la saturación cognitiva del usuario, uno de los principales desafíos en entornos urbanos complejos. 

- Si bien el sistema presenta limitaciones inherentes al uso de visión monocular y a la variabilidad del entorno real (falsos positivos, errores de OCR en vídeo, ambigüedades cromáticas o sensibilidad al movimiento de cámara), estas han sido **claramente identificadas, justificadas y contextualizadas** dentro del alcance de un prototipo funcional. En conjunto, **SafeStep** no solo valida la viabilidad técnica del enfoque propuesto, sino que establece una **base sólida, modular y extensible** para futuras mejoras, como modelos especializados, fusión de sensores o adaptación dinámica al contexto, consolidándose como una **prueba de concepto robusta, realista y centrada en la accesibilidad urbana**.